Intalações e imports

In [ ]:
pip install zenpy

In [ ]:
from zenpy import Zenpy
from zenpy.lib.api_objects import Ticket
from datetime import datetime
import pandas as pd

Credenciais

In [ ]:
zenpy_client = Zenpy(email = "", token = "", subdomain = "")

TRF

In [ ]:
# Recupere todos os tickets disponíveis
todos_tickets = zenpy_client.tickets()

contador = 0
for chamado in todos_tickets:
    contador += 1

    # Obtenha o momento de criação do ticket como um objeto de data
    momento_de_criacao = datetime.strptime(chamado.created_at, "%Y-%m-%dT%H:%M:%SZ")

    # Recupere os comentários do ticket (as respostas)
    comentarios_do_ticket = zenpy_client.tickets.comments(ticket=chamado.id)

    # Encontre o primeiro comentário público no ticket (a primeira resposta)
    momento_primeira_resposta = None
    for comentario in comentarios_do_ticket:
        if comentario.public:
            momento_primeira_resposta = datetime.strptime(comentario.created_at, "%Y-%m-%dT%H:%M:%SZ")
            break

    if momento_primeira_resposta:
        tfr_em_segundos = (momento_primeira_resposta - momento_de_criacao).total_seconds()
        print(f"Tempo para a Primeira Resposta (TFR) para o Ticket #{chamado.id}: {tfr_em_segundos} segundos")
    else:
        print(f"Nenhuma primeira resposta encontrada para o Ticket #{chamado.id}")


print(f"Total de itens processados: {contador}")


DF com todos os dados

In [ ]:
# Data de início e fim do intervalo desejado
start_time = datetime(2023, 1, 1, 0, 0, 0)
end_time = datetime(2023, 12, 31, 23, 59, 59)
params = {
    'created_from': start_time,
    'created_to': end_time,
}


tickets = zenpy_client.tickets(**params)

unique_ticket_ids = set()

# Crie uma lista de dicionários com as informações dos tickets
ticket_data = []
for ticket in tickets:

    if ticket.id in unique_ticket_ids:
        continue


    unique_ticket_ids.add(ticket.id)

    ticket_info = {
        'ID do Ticket': ticket.id,
        'Criado em': ticket.created_at,
        'Assunto': ticket.subject,
        'Status': ticket.status,
        'Prioridade': ticket.priority,
        'Tipo': ticket.type,
        'Atribuído a': ticket.assignee.name if ticket.assignee else 'Não atribuído',
        'Última Atualização': ticket.updated_at,
        'Resolvido em': ticket.updated_at if ticket.status == 'Fechado' else 'Não resolvido',
        'Grupo de Suporte': ticket.group.name if ticket.group else 'Não atribuído a grupo',
        'Organização do Solicitante': ticket.organization.name if ticket.organization else 'Não atribuído',
        'Solicitante': ticket.requester.name if ticket.requester else 'Não atribuído',
        'Solicitante E-mail': ticket.requester.email if ticket.requester else 'E-mail não fornecido',
        'Data de Vencimento': ticket.due_at,
    }

    # Calcular o TFR (Tempo para a Primeira Resposta)
    comentarios_do_ticket = zenpy_client.tickets.comments(ticket.id)
    primeiro_comentario_publico = next((comentario.created_at for comentario in comentarios_do_ticket if comentario.public), None)
    if primeiro_comentario_publico:
        tfr_em_segundos = (datetime.strptime(primeiro_comentario_publico, "%Y-%m-%dT%H:%M:%SZ") - datetime.strptime(ticket.created_at, "%Y-%m-%dT%H:%M:%SZ")).total_seconds()
    else:
        tfr_em_segundos = None

    ticket_info['TFR (Tempo para a Primeira Resposta)'] = tfr_em_segundos

    ticket_data.append(ticket_info)


df = pd.DataFrame(ticket_data)


In [ ]:
df